GEE Index Collection

In [1]:
## GEE setup
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=pYA-u02lmq5Qg-XKBkg4fjWnNx_hm9Ohk90XE-u-PJ4&tc=jZjSlqJ_7TaPr0lPCaVC6r1RuSabEKdbf--JLqM6QBQ&cc=Dx3wcdBoWq2BOXwft2mqDfVQyKuz1UViDwvQ8RpZNrI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkHoRfqgRkpAAizgKUJafeXrFFELWy6mxFQY6i63y86KKFQsiK7E3E

Successfully saved authorization token.


In [2]:
# collections
l8sr = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') ## land 8
pts = ee.FeatureCollection('projects/poulos-gee/assets/fire_ign')
l8bands = ['SR_B5', 'SR_B4', 'SR_B6', 'SR_B3']

In [3]:
# index calculations
# ndvi 8
def make_ndvi_lan8(image):
    nir = image.select('SR_B5')
    red = image.select('SR_B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return ndvi
# ndmi 8
def make_ndmi_lan8(image):
    nir = image.select('SR_B5')
    swir = image.select('SR_B6')
    ndmi = nir.subtract(swir).divide(nir.add(swir)).rename('NDMI')
    return ndmi
# ndwi 8
def make_ndwi_lan8(image):
    green = image.select('SR_B3')
    nir = image.select('SR_B5')
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    return ndwi
# cloud mask function
def cloud_mask_lan8(img):
    cloud_mask = img.select('QA_PIXEL').bitwiseAnd(ee.Number.parse('0000001111', 2)).eq(0)
    return(img.select(l8bands).updateMask(cloud_mask))

In [4]:
## cloud mask and make indexs

# clouds
l8cloud = l8sr.map(cloud_mask_lan8)
# ndvi
l8ndvi = l8cloud.map(make_ndvi_lan8)
# ndmi
l8ndmi = l8cloud.map(make_ndmi_lan8)
# ndwi
l8ndwi = l8cloud.map(make_ndwi_lan8)


In [5]:
## new functions

# buffer
def buf(col):
    pt = ee.Feature(col).buffer(15)
    return(pt)

# zonal stats
def zonal_ndvi(col):
    ign = ee.Date(col.get('ignitiondate')) # ee.Date(col.get(....))
    mon3 = ee.Date(col.get('prior3m_ign'))
    img = l8ndvi.filterDate(mon3, ign)

    def zone(imgcol):
        reg = imgcol.reduceRegion(
            reducer = ee.Reducer.mean(),
            geometry = col.geometry(),
            scale = 30)
    return ee.Feature(col.geometry(), reg)

    zstat = img.map(zone)
    return zstat



In [ ]:
# testing
# check to see if fire_ign imported correctly
ptx = pts.first()
print(ptx.getInfo())

# create a test feature
tpt = ee.Feature(ee.Geometry.Point([-112.2599178, 36.48981674])).set()

col = pts.map(buf)
ign = ee.Date(col.get('ignitiondate')) # ee.Date(col.get(....))
mon3 = ee.Date(col.get('prior3m_ign'))

img = l8ndvi.filterBounds(col.geometry()).filterDate(mon3, ign)
print(img.getInfo().first)

def zone(imgcol):
    reg = imgcol.reduceRegion(
            reducer = ee.Reducer.mean(),
            geometry = col.geometry(),
            scale = 30)
    return ee.Feature(col.geometry(), reg)

zstat = img.map(zone)
zstat.


# can we add columns to feature colletion
# add featrue tes

# def add_feat(col):
#     return(col.set('num', 1).set('char', 4))
# pts2 = pts.map(add_feat)
# pts2.first().getInfo()

# f = ptx.get('fire')
# ptx.getInfo()

# pt = ee.Feature(ptx).buffer(15)
# pt.getInfo()

# start = pt.get('prior3m_ign')
# end = pt.get('ignitiondate')
# tndvi = l8ndvi.filterDate(start, end)
# tndvi.first().getInfo()

# test to see if buffer works
# bufpts = pts.map(buf)


# z = bufpts.map(zonal_ndvi)
# z.first().getInfo()

## testing data set
#tpts = pts.limit(2)
#buftpts = tpts.map(buf)
#print('buffered')
#z = buftpts.map(zonal_ndvi)
#print('mapped')
#z






{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-111.37415099642493, 35.31043066225664]}, 'id': '00000000000000000029', 'properties': {'category': 'maroon 600', 'fire': 'maroon', 'fireID': 'AZ3530911140720190516', 'ignitiondate': '2019-05-16', 'objectid': 600, 'prior3m_ign': '2019-02-16'}}


EEException: ignored

In [ ]:
print('t')

t
